<a href="https://colab.research.google.com/github/sanhiitaa/100-days-of-deep-learning/blob/main/age_gender_non_linear_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

GPU unavialable atm, file run pending.

In [17]:
# importing the kaggle API
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

cp: cannot stat 'kaggle.jason': No such file or directory


In [18]:
# dowloading the dataset
!kaggle datasets download -d jangedoo/utkface-new

Dataset URL: https://www.kaggle.com/datasets/jangedoo/utkface-new
License(s): copyright-authors
utkface-new.zip: Skipping, found more recently modified local copy (use --force to force download)


In [ ]:
# unzipping the file
import zipfile
zip_ref=zipfile.ZipFile('/content/utkface-new.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [5]:
# importing libraries
import os
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator

In [6]:
folder_path='/content/utkface_aligned_cropped/UTKFace'

In [7]:
age=[]
gender=[]
img_path=[]
for file in os.listdir(folder_path):
  age.append(int(file.split('_')[0]))
  gender.append(int(file.split('_')[1]))
  img_path.append(file)

In [8]:
df=pd.DataFrame({'age': age,
                 'gender': gender,
                 'image': img_path})
df.head()

,age,gender,image
0,1,0,1_0_2_20161219140811232.jpg.chip.jpg
1,50,0,50_0_0_20170104213122948.jpg.chip.jpg
2,50,1,50_1_0_20170109012358091.jpg.chip.jpg
3,29,1,29_1_0_20170119164401126.jpg.chip.jpg
4,29,0,29_0_0_20170116224736533.jpg.chip.jpg


In [9]:
df.shape

(23708, 3)

In [10]:
train_df=df.sample(frac=1).iloc[:20000]
test_df=df.sample(frac=1).iloc[20000:]

In [11]:
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.2,
                                   height_shift_range=0.3,
                                   shear_range=0.4,
                                   zoom_range=0.1,
                                   horizontal_flip=True)

train_generator= train_datagen.flow_from_dataframe(train_df,
                                                   directory=folder_path,
                                                   x_col='image',
                                                   y_col=['age', 'gender'],
                                                   target_size=(200,200),
                                                   class_mode='multi_output')

test_datagen = ImageDataGenerator(rescale=1./255)

test_generator= test_datagen.flow_from_dataframe(test_df,
                                                   directory=folder_path,
                                                   x_col='image',
                                                   y_col=['age', 'gender'],
                                                   target_size=(200,200),
                                                   class_mode='multi_output')

Found 20000 validated image filenames.
Found 3708 validated image filenames.


In [12]:
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.models import Model

In [13]:
resnet= ResNet50(include_top=False,
                 input_shape=(200,200,3))

resnet.trainable=False

# setting the last layer of resnet conv_base as output layer
conv_output = resnet.layers[-1].output

# joining the output layer with flatten
flatten= Flatten()(conv_output)

# joining hidden layers to the flatten layer
dense1= Dense(512, activation='relu')(flatten)
dense2= Dense(512, activation='relu')(flatten)

dense3= Dense(512, activation='relu')(dense1)
dense4= Dense(512, activation='relu')(dense2)

output1= Dense(1, activation='linear', name='age')(dense3)
output2= Dense(1, activation='sigmoid', name='gender')(dense4)

94765736/94765736 [==============================] - 2s 0us/step


In [14]:
model=Model(inputs=resnet.input, outputs=[output1, output2])

In [15]:
model.compile(optimizer='adam',
              loss={'age': 'mae',
                    'gender': 'binary_crossentropy'},
              metrics={'age': 'mae',
                    'gender': 'accuracy'},
              loss_weights={'age':1,
                           'gender':99})

In [ ]:
model.fit(train_generator,
          batch_size=32,
          epochs=10,
          validation_data=test_generator)